In [1]:
import pandas as pd

In [2]:
import io
def load_any(file_bytes, filename):
    if filename.lower().endswith((".xlsx", ".xls")):
        return pd.read_excel(io.BytesIO(file_bytes), header=None, dtype=str)
    return pd.read_csv(io.BytesIO(file_bytes), header=None, dtype=str)



In [3]:
dff = load_any(file_bytes,"Acct Statement_8340_07122025_20.42.57.xls")

NameError: name 'file_bytes' is not defined

In [4]:
KEYWORDS = ["date", "narr", "desc", "with", "depo", "amount", "value"]

def detect_header(df):
    for i in range(min(40, len(df))):  # scan first 40 rows
        row = " ".join(str(x).lower() for x in df.iloc[i].fillna(""))
        if sum(1 for k in KEYWORDS if k in row) >= 2:
            return i
    return 0  # fallback


In [24]:
detect_header(dff)

20

In [5]:
def extract_table(file_bytes, filename):
    raw = load_any(file_bytes, filename)
    hdr = detect_header(raw)
    df = pd.read_excel(io.BytesIO(file_bytes), header=hdr, dtype=str) \
         if filename.endswith((".xls",".xlsx")) \
         else pd.read_csv(io.BytesIO(file_bytes), header=hdr, dtype=str)
    return df


In [6]:
def auto_map(df):
    cols = {c.lower(): c for c in df.columns}

    date_col = next((v for k,v in cols.items() if "date" in k), None)
    desc_col = next((v for k,v in cols.items() if "narra" in k or "desc" in k or "part" in k), None)

    # priority: withdrawal/deposit → amount
    withdraw = next((v for k,v in cols.items() if "with" in k), None)
    deposit  = next((v for k,v in cols.items() if "depo" in k), None)

    amount_col = withdraw or deposit or next((v for k,v in cols.items() if "amount" in k), None)

    return date_col, desc_col, withdraw, deposit, amount_col


In [12]:
import re

def safe_float(val):
    if val is None:
        return None

    s = str(val).strip()

    # treat ***** or ---- or NaN as null
    if re.fullmatch(r"[\*\- ]+", s):
        return None

    # remove commas/currency
    s = s.replace(",", "").replace("₹", "").replace("$", "")

    # parentheses negative values: (124.50)
    if s.startswith("(") and s.endswith(")"):
        s = "-" + s[1:-1]

    # final safety check
    if not re.fullmatch(r"-?\d+(\.\d+)?", s):
        return None

    try:
        return float(s)
    except:
        return None


In [18]:
def normalize(df):
    date_col, desc_col, withdraw, deposit, amount_col = auto_map(df)

    def compute_amt(row):
        if withdraw:
            w = safe_float(row.get(withdraw))
            if w is not None:
                return -abs(w)

        if deposit:
            d = safe_float(row.get(deposit))
            if d is not None:
                return abs(d)

        # fallback to amount column
        val = safe_float(row.get(amount_col))
        return val



    df = df.copy()
    df = df.dropna(how="all")  # drop blank rows

    out = pd.DataFrame()
    out["tx_date"] = pd.to_datetime(df[date_col], errors="coerce")
    out["description"] = df[desc_col].astype(str)

    out["amount"] = df.apply(compute_amt, axis=1)

    # drop rows that still have no valid amount or no valid date
    out = out.dropna(subset=["tx_date", "amount"])

    out["direction"] = out["amount"].apply(lambda x: "debit" if x < 0 else "credit")

    return out





In [19]:
def parse_statement(file_bytes, filename):
    df = extract_table(file_bytes, filename)
    df = normalize(df)
    return df[["tx_date", "description", "amount", "direction"]]


In [20]:
with open("Acct Statement_8340_07122025_20.42.57.xls", "rb") as f:
    file_bytes = f.read()


In [31]:
df_final = parse_statement(file_bytes,"Acct Statement_8340_07122025_20.42.57.xls").head(100)

C:\Users\devar\AppData\Local\Temp\ipykernel_19532\263978420.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  out["tx_date"] = pd.to_datetime(df[date_col], errors="coerce")
